El objetivo de este notebook es demostrar como se puede evaluar la <b>similitud de documentos</b> utilizando python

<b>Referencias</b>
<ul>
<li>
http://blog.christianperone.com/2011/09/machine-learning-text-feature-extraction-tf-idf-part-i/
</li>
<li>
http://blog.christianperone.com/2011/10/machine-learning-text-feature-extraction-tf-idf-part-ii/
</li>
<li>
http://blog.christianperone.com/2013/09/machine-learning-cosine-similarity-for-vector-space-models-part-iii/
</li>

## Leer el archivo csv con jobs

In [4]:
#Leer el archivo con descripciones de Bumeran
import pandas as pd
job_raw = pd.read_csv("bumall15oct.csv", encoding='cp1252')

In [5]:
#visualizar la carga
job_raw.head()

,FECHA_SCRAP,CATEGORIA,FUNCION,EMPRESA,PUESTO,DESCRIPCION,URL,PAGE,SITE,FECHA_PUB
0,15/10/2018 19:47,NaN,NaN,GSS CALL CENTER,Ejecutivo Ventas CALL CENTER C/EXP PRESENCIAL ...,TE INVITAMOS A SER PARTE DE NUESTRA FAMILIA GS...,http://www.bumeran.com.pe/empleos/ejecutivo-ve...,http://www.bumeran.com.pe/empleos-peru.html,Bumeran,15/10/2018
1,15/10/2018 19:47,NaN,NaN,PANDERO S.A. EAFC,Jefe de Selección del Talento,Principales Funciones: - Responsable de los pr...,http://www.bumeran.com.pe/empleos/jefe-de-sele...,http://www.bumeran.com.pe/empleos-peru.html,Bumeran,14/10/2018
2,15/10/2018 19:47,NaN,NaN,ANCRO,Administrador de Cuenta Comercial - Villa El S...,Administrador de Cuenta Comercial - Zona Sur E...,http://www.bumeran.com.pe/empleos/administrado...,http://www.bumeran.com.pe/empleos-peru.html,Bumeran,12/10/2018
3,15/10/2018 19:47,NaN,NaN,PANDERO S.A. EAFC,Analista Sr. Comunicación Organizacional,Funciones: - Generar la estrategia y el plan a...,http://www.bumeran.com.pe/empleos/analista-sr....,http://www.bumeran.com.pe/empleos-peru.html,Bumeran,14/10/2018
4,15/10/2018 19:47,NaN,NaN,Talent 360,Mecanico Aeronautico - Latam Airlines Chile,La Gerencia de Mantenimiento Mayor de Latam Ar...,http://www.bumeran.com.pe/empleos/mecanico-aer...,http://www.bumeran.com.pe/empleos-peru.html,Bumeran,12/10/2018


In [6]:
#cargar la series con descripciones del puesto
desc = job_raw.DESCRIPCION
desc

0        TE INVITAMOS A SER PARTE DE NUESTRA FAMILIA GS...
1        Principales Funciones: - Responsable de los pr...
2        Administrador de Cuenta Comercial - Zona Sur E...
3        Funciones: - Generar la estrategia y el plan a...
4        La Gerencia de Mantenimiento Mayor de Latam Ar...
5        PACIFICO SEGUROS, empresa líder en el mercado ...
6        Nestlé Perú , la empresa líder en Nutrición, S...
7        Por encargo de nuestro cliente, Empresa Líder ...
8        ¡Ingresa a Atento y empieza a cumplir tus sueñ...
9        LA MEJOR FORMA DE GANAR ALTOS INGRESOS “Te con...
10       Requisitos: ü Contar con experiencia de 5 años...
11       Nos encontramos en busca de consultores parala...
12       En Graña y Montero nos encontramos en búsqueda...
13       CEMEX PERU es una empresa dedicada a la comerc...
14       Importante empresa de Call Center con más de 1...
15       TE INVITAMOS A SER PARTE DE NUESTRA FAMILIA GS...
16       MAPFRE , primer grupo asegurador de España y L.

In [7]:
#ver el 2do registro a detalle
desc[1]

'Principales Funciones: - Responsable de los procesos de selección de mandos medios y gerenciales. - Responsable de la mejora continua en los procesos de reclutamiento administrativo. - Gestionar los procesos de desarrollo organizacional y líneas de carrera. Requisitos: - Bachiller/ Titulado en psicología. (Mandatorio) - Experiencia mínima de 10 años en selección, especialmente gerencialy mandos medios. - Experiencia en headhunting. - Experiencia en evaluación de candidatos (psicológicas y por competencias)   El contenido de este aviso es de propiedad del anunciante. Los requisitos de la posición son definidos y administrados por el anunciante sin que Bumeran sea responsable por ello.'

Con esto observamos que hay que hacer limpieza de texto para eliminar <b>caracteres no deseados</b>

## Data Cleaning

In [8]:
# Probar con expresiones regulares
import re
#Solo quiero letras
letras = re.sub("[^a-zA-ZáóéíúñÑ]", " ", desc[1] )  
print (letras)

Principales Funciones    Responsable de los procesos de selección de mandos medios y gerenciales    Responsable de la mejora continua en los procesos de reclutamiento administrativo    Gestionar los procesos de desarrollo organizacional y líneas de carrera  Requisitos    Bachiller  Titulado en psicología   Mandatorio    Experiencia mínima de    años en selección  especialmente gerencialy mandos medios    Experiencia en headhunting    Experiencia en evaluación de candidatos  psicológicas y por competencias    El contenido de este aviso es de propiedad del anunciante  Los requisitos de la posición son definidos y administrados por el anunciante sin que Bumeran sea responsable por ello 


El resultado es el deseado, ya que nos concentraremos en <b>palabras</b>

In [9]:
# Pasar a minusculas
minusculas = letras.lower()        
palabras = minusculas.split()   


In [10]:
# Excluir los llamados stopwords
import nltk
from nltk.corpus import stopwords 
print (stopwords.words("spanish")) 

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosostros', 'vosostras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 

In [11]:
# Quitar los stopwords de la lista
palabras = [w for w in palabras if not w in stopwords.words("spanish")]
print (palabras)

['principales', 'funciones', 'responsable', 'procesos', 'selección', 'mandos', 'medios', 'gerenciales', 'responsable', 'mejora', 'continua', 'procesos', 'reclutamiento', 'administrativo', 'gestionar', 'procesos', 'desarrollo', 'organizacional', 'líneas', 'carrera', 'requisitos', 'bachiller', 'titulado', 'psicología', 'mandatorio', 'experiencia', 'mínima', 'años', 'selección', 'especialmente', 'gerencialy', 'mandos', 'medios', 'experiencia', 'headhunting', 'experiencia', 'evaluación', 'candidatos', 'psicológicas', 'competencias', 'contenido', 'aviso', 'propiedad', 'anunciante', 'requisitos', 'posición', 'definidos', 'administrados', 'anunciante', 'bumeran', 'responsable', 'ello']


In [12]:
#juntar todo 
resultado = " ".join(palabras)
print (resultado)

principales funciones responsable procesos selección mandos medios gerenciales responsable mejora continua procesos reclutamiento administrativo gestionar procesos desarrollo organizacional líneas carrera requisitos bachiller titulado psicología mandatorio experiencia mínima años selección especialmente gerencialy mandos medios experiencia headhunting experiencia evaluación candidatos psicológicas competencias contenido aviso propiedad anunciante requisitos posición definidos administrados anunciante bumeran responsable ello


## Aplicar el cleaning a todos los registros 

In [13]:
def desc_to_words(raw):
    #
    # 1. Solo letras     
    letras = re.sub("[^a-zA-ZáóéíúñÑ]", " ", raw) 
    # 2. convertir a minusculas
    words = letras.lower().split()                             
    #
    # 3. convertir a set ya que es más rapido
    stops = set(stopwords.words("spanish"))                  
    # 
    # 4. Quitar stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 5. Unir las palabras, 
    # Retornar resultado.
    return( " ".join( meaningful_words ))   

In [14]:
# Sacar el numero de registros
num_filas = desc.size
desc_limpio = []
for i in range(0, num_filas):
    desc_limpio.append(desc_to_words(desc[i]))
print ("COMPLETADO")

COMPLETADO


In [15]:
desc_limpio[13]

'cemex peru empresa dedicada comercialización distribución cementos oportunidad encontramos búsqueda mejor talento siguiente posición tecnico mec nico mantenimiento requisitos egresado bachiller carrera técnicas mecánica mantenimiento mecánico máquinas herramientas experiencia haber trabajado empresas industriales experiencia trabajo horarios rotativos manejo office nivel básico manejo inglés conocimientos conocimientos máquinas herramientas torno fresa conocimientos soldadura eléctrica conocimientos seguridad industrial autocad funciones realizar trabajos mecanizado máquinas herramientas torno cepillo fresador realizar trabajos mecánicos mantenimiento preventivos correctivos según plan mantenimiento realizar trabajos lubricación máquinas según plan mantenimiento realizar trabajos soldaduras armado estructuras metálicas realizar mantenimiento sistemas mecánicos máquinas junto técnico mecánico mantenimiento funciones dispuestas jefatura beneficios grato ambiente laboral eps cubierta ben

In [16]:
#convertir a pandas series
desc_limpio = pd.Series(desc_limpio)
print (desc_limpio)

0        invitamos ser parte familia gss crecimiento pi...
1        principales funciones responsable procesos sel...
2        administrador cuenta comercial zona sur egresa...
3        funciones generar estrategia plan anual comuni...
4        gerencia mantenimiento mayor latam arilines ch...
5        pacifico seguros empresa líder mercado asegura...
6        nestlé perú empresa líder nutrición salud bien...
7        encargo cliente empresa líder mercado suminist...
8        ingresa atento empieza cumplir sueños trabaja ...
9        mejor forma ganar altos ingresos convertimos d...
10       requisitos contar experiencia años desempeñánd...
11       encontramos busca consultores parala realizaci...
12       graña montero encontramos búsqueda mejor talen...
13       cemex peru empresa dedicada comercialización d...
14       importante empresa call center años trayectori...
15       invitamos ser parte familia gss grupo g s s gl...
16       mapfre primer grupo asegurador españa latinoam.

## TF-IDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(desc_limpio)
print (tfidf_matrix.shape)

(12659, 32282)


## CV

In [17]:
#cargar CV en word
import docx

In [18]:
def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [19]:
read_word = getText('CV_Abogado.docx')

In [20]:
# Probar con expresiones regulares
import re
#Solo quiero letras
letrascv = re.sub("[^a-zA-ZáóéíúñÑ]", " ", read_word)  

In [21]:
# Pasar a minusculas
minusculascv = letrascv.lower()        
palabrascv = minusculascv.split()   

In [22]:
# Quitar los stopwords de la lista
import nltk
from nltk.corpus import stopwords 
palabrascv = [w for w in palabrascv if not w in stopwords.words("spanish")]

In [23]:
#juntar todo 
resultadocv = " ".join(palabrascv)

In [24]:
import pandas as pd
#convertir a pandas series
desc_limpio_cv = pd.Series(resultadocv)
print (desc_limpio_cv)

0    ejemplo cv alberto sala gonzalez c heroes alca...
dtype: object


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf_vectorizer2 = TfidfVectorizer()
tfidf_matrix2 = tfidf_vectorizer.transform(desc_limpio_cv)
print (tfidf_matrix2.shape)

(1, 32282)


## Cosine Similarity

In [26]:
#Comparar el 1er documento al resto
from sklearn.metrics.pairwise import cosine_similarity
#res = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix, True)
res = cosine_similarity(tfidf_matrix2, tfidf_matrix, True)
res = sorted(res[0], reverse=True)
res

[0.14615387608650793,
 0.1353608781510296,
 0.12852623949293454,
 0.12702582716766092,
 0.1267973360156066,
 0.1252110509575092,
 0.1232360465523036,
 0.12254728719239152,
 0.1133320487323399,
 0.10951353804215425,
 0.10415377060937309,
 0.10371399603857648,
 0.10343938114034291,
 0.1028139577022246,
 0.1020037245117951,
 0.10171288426218565,
 0.10044609096030302,
 0.10008499020306391,
 0.09366239723338912,
 0.09183114956615583,
 0.09183114956615583,
 0.09183114956615583,
 0.09151218387610274,
 0.0903393602197102,
 0.09017218682661406,
 0.08988103292223827,
 0.0895853911435347,
 0.08806321980863811,
 0.08775899925474044,
 0.08595610648888853,
 0.08437073023795058,
 0.0840019890876622,
 0.08357762528145898,
 0.08271561796590438,
 0.08186689731207068,
 0.08175969220780259,
 0.08164801351248026,
 0.08048529822829456,
 0.08026948866925454,
 0.08026948866925454,
 0.07921320414628127,
 0.07866884672560633,
 0.07817142793683815,
 0.07806695098388669,
 0.07757994001224883,
 0.07738791148835548

In [27]:
#obtener el nombre del documento y crear un dataframe
res = cosine_similarity(tfidf_matrix2, tfidf_matrix, True)
res = res[0]

In [28]:
res

array([0.010347  , 0.01858795, 0.01216519, ..., 0.02180468, 0.00428587,
       0.01097224])

In [29]:
size = len(res)
#crear el dataframe
job_simil = pd.DataFrame(columns=('ID', 'Puesto', 'Similitud'))
#job_simil
i = int()
#llenar los datos
for i in range(0, size):
    job_simil.loc[i] = [i+1, job_raw['PUESTO'][i],res[i]]
#hacer un sort por similitud    
sorted_job = job_simil.sort_values(['Similitud'], ascending=False)

In [30]:
sorted_job

,ID,Puesto,Similitud
3852,3853,Abogado - Asesor jurídico de control interno,0.146154
3483,3484,Secretaria Bilingüe,0.135361
7270,7271,DOCENTE TP–CURSO DERECHO DE LA PROPIEDAD INTEL...,0.128526
9189,9190,PROYECTISTA ( ARQUITECTO / ARQUITECTA ),0.127026
7147,7148,ABOGADO SENIOR /ASESOR LEGAL - DERECHO PENAL O...,0.126797
8975,8976,EJECUTIVO (A) DE COBRANZAS,0.125211
9116,9117,Abogado Especializado en Derecho Laboral,0.123236
5949,5950,Asistente Legal,0.122547
3757,3758,Asistente de Asesoría Legal Interna - Sede Iqu...,0.113332
8164,8165,Bachiller de Derecho,0.109514
